# タスク 2: 質問応答に Amazon Bedrock を使用する

このノートブックでは、Bedrock Nova Lite モデルを使用して、関連するすべてのコンテキストを含むリクエストをモデルに送信し、応答を求めることで、クエリに対する情報応答を提供する方法を学びます。これにより、事前にドキュメントを準備して索引付けしなくても、モデルが質問に対して事実に基づく回答を返すという課題に対処できます。

このノートブックは、**検索拡張生成 (RAG)** をシミュレートしますが、実際には RAG を使用していません。このアプローチは、短いドキュメントやシングルトンのアプリケーションでは有効ですが、モデルに送信されるプロンプトに大量の企業ドキュメントすべてが収まらないような企業レベルの質問応答には対応できない場合があります。

**質問応答 (QA)** は、自然言語で提示された事実に基づくクエリに対する回答抽出に関与する重要なタスクです。通常、QA システムは、構造化データまたは非構造化データを含むナレッジベースに対してクエリを処理し、正確な情報を含む応答を生成します。高い精度を確保することは、特にエンタープライズのユースケースにおいて、有用で信頼性が高く、信用できる質問応答システムを開発するための鍵となります。


## シナリオ

AnyCompany で、質問応答モデルに質問して、同社が製造する特定の車両モデルのタイヤ交換に関する情報を提供する状況をモデル化してみます。まず、「ゼロショット」アプローチを使用してモデルにクエリを実行し、トレーニングデータだけに基づいて関連性の高い回答が得られるかどうかを確認します。

しかし、このモデルが「ハルシネーション」を起こして、より一般的な答えを出しているようだと気付きます。それは、架空の車両モデルを試しても同様の応答が得られることからわかります。つまり、モデルのトレーニングを拡張するため、Example Company の実際の車両マニュアルを追加して、各モデルのタイヤの詳細を説明する必要があるということです。

このラボでは、外部データを使用せずに、このような「検索拡張生成」 (RAG) アプローチをシミュレートします。AnyCompany Model Z の車両のタイヤ交換方法を説明した詳細なマニュアルの抜粋を提供します。今度はモデルがコンテキストに沿ったこのサンプルコンテンツを活用して、カスタマイズされた正確な答えを出すことができるかどうかをテストします。

## タスク 2.1: 環境の設定

このタスクでは、環境を設定します。

In [ ]:
# Create a service client by name using the default session.
import json
import os
import boto3

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

# Configure the modelId and request/response format
modelId = "amazon.nova-lite-v1:0"
accept = "application/json"
contentType = "application/json"

# Imports to display markdown outputs
from IPython.display import display_markdown, Markdown, clear_output

In [ ]:
"""
Nova Lite Adapter:

This code block contains helper functions for using Nova Lite.
"""

def format_for_nova_lite(prompt_text):
    """Format the prompt for Nova Lite's expected message structure."""
    return {
        "messages": [
            {
                "role": "user",
                "content": [{"text": prompt_text}]
            }
        ],
        "inferenceConfig": {
            "maxTokens": 2048,
            "temperature": 0,
            "topP": 0.9
        }
    }

def parse_nova_lite_response(response_body):
    """Parse the response from Nova Lite."""
    if 'output' in response_body and 'message' in response_body['output']:
        message = response_body['output']['message']
        if 'content' in message and isinstance(message['content'], list):
            # Extract text from each content item
            texts = []
            for content_item in message['content']:
                if isinstance(content_item, dict) and 'text' in content_item:
                    texts.append(content_item['text'])
            return ' '.join(texts)
    
    # Fallback if the response format is different
    return str(response_body)

## タスク 2.2: モデルの知識を使用した質問応答
このセクションでは、Bedrock サービスが提供するモデルを使用して、トレーニングフェーズで得た知識に基づいて質問への回答を試みます。

このタスクでは、Amazon Bedrock クライアントの invoke_model() メソッドを使用します。このメソッドを使用するための必須パラメータは、Amazon Bedrock モデルの ARN を表す modelId と、タスクのプロンプトである body です。

body プロンプトは、選択した基盤モデルプロバイダーによって異なります。これについては、以下で詳しく説明します。

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

Bedrock サービスが提供するモデルを使用して、トレーニングフェーズで得た知識に基づいて質問への回答を試みます。

In [ ]:
prompt_data = """あなたは有能なアシスタントです。質問には簡潔に答えてください。答えに自信がない場合は、「わかりません」と回答して下さい。

Question: AnyCompany AC8 のパンクしたタイヤを修理するにはどうすればいいですか？
Answer:"""

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** 以降のタスクでコードを実行すると、*ThrottlingException* メッセージが表示され、再試行される場合があります。コードには、失敗したリクエストをエクスポネンシャルバックオフで自動的に再試行する堅牢なエラー処理が組み込まれています。これはサービスクォータを使用する際の正常な動作であり、本番環境に対応したアプリケーションが API の制限をどのように処理すべきかを示しています。

## タスク 2.3: JSON 本文を渡すことでモデルを呼び出して応答を生成する

In [ ]:
try:
    # Create the body from the prompt
    body = format_for_nova_lite(prompt_data)

    # Invoke the model
    response = bedrock_client.invoke_model(
        body=json.dumps(body), 
        modelId=modelId, 
        accept=accept, 
        contentType=contentType
    )
    
    # Read from the response stream
    response_body = json.loads(response.get('body').read())

    # Parse the output
    outputText = parse_nova_lite_response(response_body)

    # Display the output text in markdown format
    display_markdown(Markdown(outputText))
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")


このモデルでは、パンクしたタイヤを交換するプロセスの概要を説明する回答が得られますが、同じ説明はどの車にも当てはまる可能性があります。残念ながら、これはスペアタイヤのない AnyCompany AC8 にとっては正しい答えではありません。これは、車のタイヤ交換に関する指示を含むデータでモデルがトレーニングされているためです。

まったく架空のブランドとモデルの車 (例えば Amazon Tirana) で同じ質問をしてみると、この問題の別の例を見ることができます。

In [ ]:
# Usage
prompt_data = """Amazon Tirana のパンクしたタイヤを修理するにはどうすればいいですか?
Answer: パンクしたタイヤを修理する手順は次のとおりです:
"""

try:
    # Create the body from the prompt
    body = format_for_nova_lite(prompt_data)

    # Invoke the model
    response = bedrock_client.invoke_model(
        body=json.dumps(body), 
        modelId=modelId, 
        accept=accept, 
        contentType=contentType
    )
    
    # Read from the response stream
    response_body = json.loads(response.get('body').read())

    # Parse the output
    outputText = parse_nova_lite_response(response_body)

    # Display the output text in markdown format
    display_markdown(Markdown(outputText))
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")

このプロンプトの質問の場合、モデルは現実的な答えを出すことができません。

この問題を解決し、該当する車種に有効な特定の指示に基づいてモデルに回答させるには、プロンプトの一部として追加のナレッジベースを提供することで、モデルの知識をその場で増やすことができます。

これを使ってアプリケーションを改善する方法を見てみましょう。

以下を AnyCompany AC8 のマニュアルからの抜粋であると仮定します (実際は本物のマニュアルではありませんが、そのようなものとして扱います)。このドキュメントは短いので、Nova Lite のコンテキストウィンドウに全体が収まります。

```plain
タイヤとタイヤの空気圧:

タイヤは黒いゴムでできており、車のホイールに取り付けられています。タイヤは、運転、コーナリング、ブレーキングに必要なグリップを提供します。考慮すべき 2 つの重要な要素は、タイヤの空気圧とタイヤの摩耗です。これらは、車の性能とハンドリングに影響を与える可能性があります。

推奨タイヤ空気圧の確認場所:

推奨タイヤ空気圧の仕様は、車の運転席側 B ピラーにある空気圧ラベルに記載されています。または、車のマニュアルを参照してこの情報を入手することもできます。推奨タイヤ空気圧は、速度、乗員数、または車内の最大荷重によって異なる場合があります。

タイヤの空気圧の調整:

タイヤ空気圧を確認するときは、タイヤが冷えているときに行うことが重要です。つまり、車を少なくとも 3 時間放置して、タイヤが周囲温度と同じ温度になるようにします。

タイヤの空気圧を調整するには:

車の推奨タイヤ空気圧を確認します。

エアポンプの指示に従って、タイヤを適切な空気圧に調整します。
車両のセンターディスプレイで、「車の状態」アプリを開きます。
「タイヤ空気圧」タブに移動します。
「空気圧の調整」オプションを押して、アクションを確認します。
タイヤ空気圧を調整するには、30 km/h 以上の速度で車を数分間運転します。

注: 場合によっては、タイヤ空気圧に関する警告記号やメッセージを消すために 15 分以上運転する必要があることがあります。警告が消えない場合は、タイヤを冷ましてから上記の手順を繰り返します。

パンク:

運転中にタイヤがパンクした場合は、タイヤモビリティキットを使用して一時的にパンクを塞ぎ、タイヤを再び膨らませることができます。このキットは通常、車両の荷物スペースの裏地の下に保管されています。

タイヤモビリティキットの使用手順:

車両のテールゲートまたはトランクを開きます。
荷物スペースの裏地を持ち上げて、タイヤモビリティキットにアクセスします。
タイヤモビリティキットに付属の説明書に従って、タイヤのパンクを塞ぎます。
キットを使用した後は、必ず元の場所にしっかりと戻してください。
使用済みのシーラントボトルの廃棄と交換については、AnyCompany または適切なサービスにお問い合わせください。

タイヤ モビリティ キットは一時的な解決策であり、最高時速 80 km/h で最大 10 分または 8 km (いずれか早い方) 走行できるように設計されていることに注意してください。パンクしたタイヤはできるだけ早く交換するか、専門家に修理してもらうことをお勧めします。
```

In [ ]:
context = """タイヤとタイヤの空気圧:

タイヤは黒いゴムでできており、車のホイールに取り付けられています。タイヤは、運転、コーナリング、ブレーキングに必要なグリップを提供します。考慮すべき 2 つの重要な要素は、タイヤの空気圧とタイヤの摩耗です。これらは、車の性能とハンドリングに影響を与える可能性があります。

推奨タイヤ空気圧の確認場所:

推奨タイヤ空気圧の仕様は、車の運転席側 B ピラーにある空気圧ラベルに記載されています。または、車のマニュアルを参照してこの情報を入手することもできます。推奨タイヤ空気圧は、速度、乗員数、または車内の最大荷重によって異なる場合があります。

タイヤの空気圧の調整:

タイヤ空気圧を確認するときは、タイヤが冷えているときに行うことが重要です。つまり、車を少なくとも 3 時間放置して、タイヤが周囲温度と同じ温度になるようにします。

タイヤの空気圧を調整するには:

車の推奨タイヤ空気圧を確認します。

エアポンプの指示に従って、タイヤを適切な空気圧に調整します。
車両のセンターディスプレイで、「車の状態」アプリを開きます。
「タイヤ空気圧」タブに移動します。
「空気圧の調整」オプションを押して、アクションを確認します。
タイヤ空気圧を調整するには、30 km/h 以上の速度で車を数分間運転します。

注: 場合によっては、タイヤ空気圧に関する警告記号やメッセージを消すために 15 分以上運転する必要があることがあります。警告が消えない場合は、タイヤを冷ましてから上記の手順を繰り返します。

パンク:

運転中にタイヤがパンクした場合は、タイヤモビリティキットを使用して一時的にパンクを塞ぎ、タイヤを再び膨らませることができます。このキットは通常、車両の荷物スペースの裏地の下に保管されています。

タイヤモビリティキットの使用手順:

車両のテールゲートまたはトランクを開きます。
荷物スペースの裏地を持ち上げて、タイヤモビリティキットにアクセスします。
タイヤモビリティキットに付属の説明書に従って、タイヤのパンクを塞ぎます。
キットを使用した後は、必ず元の場所にしっかりと戻してください。
使用済みのシーラントボトルの廃棄と交換については、AnyCompany または適切なサービスにお問い合わせください。

タイヤ モビリティ キットは一時的な解決策であり、最高時速 80 km/h で最大 10 分または 8 km (いずれか早い方) 走行できるように設計されていることに注意してください。パンクしたタイヤはできるだけ早く交換するか、専門家に修理してもらうことをお勧めします。"""

##### 次に、抜粋全体を質問と一緒にモデルに渡します。

In [ ]:
question = "AnyCompany AC8 のパンクしたタイヤを修理するにはどうすればいいですか？"
prompt_data = f"""## の間に提供された情報のみに基づいて質問に答え、ステップバイステップのガイドを提供します。
#
{context}
#

Question: {question}
Answer:"""

### タスク 2.4: boto3 経由でモデルを呼び出して応答を生成する

In [ ]:
try:
    # Create the body from the prompt
    body = format_for_nova_lite(prompt_data)

    # Invoke the model
    response = bedrock_client.invoke_model(
        body=json.dumps(body), 
        modelId=modelId, 
        accept=accept, 
        contentType=contentType
    )
    
    # Read from the response stream
    response_body = json.loads(response.get('body').read())

    # Parse the output
    outputText = parse_nova_lite_response(response_body)

    # Display the output text in markdown format
    display_markdown(Markdown(outputText))
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")

モデルがコンテキストを理解して関連性の高い回答を生成するまでに時間がかかります。そのため、ユーザーは応答を数秒間待たなければならず、ユーザーエクスペリエンスが低下する可能性があります。

Bedrock は、モデルがトークンを生成すると同時にサービスが出力を生成するストリーミング機能もサポートしています。これをどのように実装できるかを次に例で示します。

In [ ]:
try:
    # Create the body from the prompt
    body = format_for_nova_lite(prompt_data)

    # Invoke the model
    response = bedrock_client.invoke_model_with_response_stream(
        body=json.dumps(body), 
        modelId=modelId, 
        accept=accept, 
        contentType=contentType
    )

    stream = response.get('body')
    output = []
    
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            # Extract text from Nova Lite's response format
            if 'contentBlockDelta' in chunk_obj and 'delta' in chunk_obj['contentBlockDelta']:
                text = chunk_obj['contentBlockDelta']['delta'].get('text', '')
                clear_output(wait=True)
                output.append(text)
                display_markdown(Markdown(''.join(output)))
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")

応答では、タイヤの交換方法が手順を追って要約されています。

これで、検索拡張生成 (RAG)、つまり拡張プロセスを活用して、提供された特定のコンテキストと情報に合わせて精選された応答を生成する方法を学びました。

### 自分でやってみよう
- プロンプトを特定のユースケースに合わせて変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えてみて、サービスのレイテンシーと応答性を理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を引き出します。

### クリーンアップ

このノートブックを完了しました。ラボの次の部分に進むために、以下を実行してください。

- このノートブックファイルを閉じて、**まとめ**に進みます。